# 🛡️ Mosaic Protocol - Vulnerability Classifier V3

## Golden Test Set Strategy
- **TRAIN** on 31K noisy V1 data (Kaggle)
- **TEST** on 145 expert-labeled SmartBugs (truth)

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
OUTPUT_DIR = '/content/drive/MyDrive/mosaic-ml'
os.makedirs(OUTPUT_DIR, exist_ok=True)

!pip install xgboost scikit-learn imbalanced-learn matplotlib seaborn --quiet
print("✅ Ready")

In [ ]:
import json, gzip, numpy as np
from pathlib import Path

# Load hybrid dataset (train on noisy, test on golden)
DATA_DIR = '/content/drive/MyDrive/mosaic-ml'
data_files = sorted(Path(DATA_DIR).glob('hybrid_dataset_*.json*'))
if not data_files:
    raise FileNotFoundError(f"Upload hybrid_dataset_*.json.gz to {DATA_DIR}")

data_file = data_files[-1]
print(f"📂 Loading: {data_file.name}")

with gzip.open(data_file, 'rt', encoding='utf-8') if str(data_file).endswith('.gz') else open(data_file) as f:
    data = json.load(f)

print(f"""\n📊 Dataset:
   Train: {data['metadata']['trainSamples']:,} (noisy)
   Golden Test: {data['metadata']['testSamples']} (expert)
   Features: {data['metadata']['featureCount']}
""")

In [ ]:
def to_arrays(samples):
    X = np.array([s['features'] for s in samples], dtype=np.float32)
    y = np.array([s['label'] for s in samples], dtype=np.int32)
    return X, y

X_train, y_train = to_arrays(data['train'])
X_val, y_val = to_arrays(data['validation'])
X_golden, y_golden = to_arrays(data['goldenTest'])

print(f"Train: {X_train.shape} (safe:{(y_train==0).sum()}, vuln:{(y_train==1).sum()})")
print(f"Val: {X_val.shape}")
print(f"Golden Test: {X_golden.shape} (safe:{(y_golden==0).sum()}, vuln:{(y_golden==1).sum()})")

feature_names = data['metadata']['featureNames']

In [ ]:
# Apply SMOTE to training data
from imblearn.over_sampling import SMOTE

print(f"Before SMOTE: safe={(y_train==0).sum()}, vuln={(y_train==1).sum()}")
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
print(f"After SMOTE: safe={(y_train_balanced==0).sum()}, vuln={(y_train_balanced==1).sum()}")

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=150,
    max_depth=5,
    learning_rate=0.08,
    eval_metric='aucpr',
    random_state=42,
)

print("🚀 Training on 31K noisy data...")
model.fit(X_train_balanced, y_train_balanced, eval_set=[(X_val, y_val)], verbose=30)
print("✅ Training complete!")

## 🎯 Golden Test Evaluation (Truth)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluate on GOLDEN test (expert labels)
y_golden_pred = model.predict(X_golden)
y_golden_proba = model.predict_proba(X_golden)[:, 1]

print("🎯 GOLDEN TEST EVALUATION (Expert Labels):")
print("="*50)
print(classification_report(y_golden, y_golden_pred, target_names=['Safe', 'Vulnerable']))

if len(np.unique(y_golden)) > 1:
    roc_auc = roc_auc_score(y_golden, y_golden_proba)
    print(f"\n🎯 ROC-AUC on Golden Set: {roc_auc:.4f}")

In [ ]:
# Confusion Matrix on Golden
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_golden, y_golden_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Safe', 'Vuln'],
            yticklabels=['Safe', 'Vuln'])
plt.title('Golden Test Confusion Matrix')
plt.ylabel('Actual (Expert)')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/golden_confusion_matrix.png")
plt.show()

In [ ]:
# PR Curve on Golden
precision, recall, thresholds = precision_recall_curve(y_golden, y_golden_proba)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, 'g-', linewidth=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall on Golden Expert Set')
plt.grid(True, alpha=0.3)
plt.savefig(f"{OUTPUT_DIR}/golden_pr_curve.png")
plt.show()

# Find optimal threshold
target_recall = 0.85
idx = np.where(recall >= target_recall)[0]
if len(idx) > 0:
    best_idx = idx[-1]
    optimal_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
    print(f"\n🎯 For {target_recall:.0%} recall: threshold={optimal_threshold:.3f}, precision={precision[best_idx]:.3f}")
else:
    optimal_threshold = 0.3
    best_idx = 0

In [ ]:
# Feature Importance
importances = model.feature_importances_
indices = np.argsort(importances)[::-1][:15]

plt.figure(figsize=(12, 6))
plt.bar(range(15), importances[indices])
plt.xticks(range(15), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.title('Top 15 Features (Hybrid Model)')
plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/feature_importance_v3.png")
plt.show()

print("\n🔝 Top 10 Features:")
for i, idx in enumerate(indices[:10]):
    print(f"   {i+1}. {feature_names[idx]}: {importances[idx]:.4f}")

In [ ]:
import joblib

# Save model
model_path = f"{OUTPUT_DIR}/vulnerability_classifier_v3.pkl"
joblib.dump(model, model_path)
print(f"✅ Saved: {model_path}")

# Save metadata
metadata = {
    'model_name': 'vulnerability_classifier_v3',
    'strategy': 'golden_test_set',
    'created_at': str(np.datetime64('now')),
    'feature_names': feature_names,
    'optimal_threshold': float(optimal_threshold),
    'golden_metrics': {
        'roc_auc': float(roc_auc) if 'roc_auc' in dir() else 0,
        'recall_at_threshold': float(recall[best_idx]) if len(idx) > 0 else 0,
        'precision_at_threshold': float(precision[best_idx]) if len(idx) > 0 else 0,
    },
    'train_samples': int(len(y_train_balanced)),
    'golden_test_samples': int(len(y_golden)),
}

with open(f"{OUTPUT_DIR}/model_metadata_v3.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n📋 Metadata saved. Key metrics:")
print(f"   Threshold: {optimal_threshold:.3f}")
print(f"   Golden ROC-AUC: {metadata['golden_metrics']['roc_auc']:.4f}")

## ✅ Complete!

**Key insight:** The Golden Test metrics are your TRUE accuracy.

If Golden PR curve looks good (not diagonal), your model learned real patterns despite noisy training data.